In [2]:
import cobra.io

modelo = cobra.io.read_sbml_model("C:\\Users\\lucia\\Desktop\\DielModels\\reconstruction_results\\MODEL1507180028\\results_troppo\\DielModel\\reconstructed_models\\Diel_Model_after_complete_pipeline.xml")

In [3]:
for metabolite in modelo.metabolites.query('hv'):
    if "Night" in metabolite.compartment:
        print(metabolite.reactions)

frozenset({<Reaction Ex16_Night at 0x226f559d700>, <Reaction REner01_p_Night at 0x226f4a88850>})


In [4]:
from cobra.flux_analysis import pfba

In [5]:
modelo.objective_direction = "max"
solution = pfba(modelo)

### Reação Fotossíntese

In [7]:
modelo.reactions.get_by_id('REner01_p_Night')

Reaction identifier,REner01_p_Night
Name,R_REner01_p
Memory address,0x226f4a88850
Stoichiometry,3.0 S_ADP_p[C_p]_Night + 2.0 S_H2O_p[C_p]_Night + 2.0 S_NADP_43__p[C_p]_Night + 3.0 S_Orthophosphate_p[C_p]_Night + 8.0 S_hv_p[C_p]_Night --> 3.0 S_ATP_p[C_p]_Night + 2.0 S_H_43__p[C_p]_Night + 2.0... 3.0 ADP_p Night + 2.0 H2O_p Night + 2.0 NADP+ Night + 3.0 Orthophosphate_p Night + 8.0 hv_p Night --> 3.0 ATP_p Night + 2.0 H+_p Night + 2.0 NADPH Night + Oxygen_p Night
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
solution.fluxes['REner01_p_Day']

125.0

In [12]:
solution.fluxes['REner01_p_Night']

0.0

### Reações Ciclo de Calvin

- Rubisco + CO2

In [14]:
modelo.reactions.get_by_id('R00024_p_Day')

Reaction identifier,R00024_p_Day
Name,ribulose bisphosphate carboxylase small chain 1A / RuBisCO small subunit 1A (RBCS-1A) (ATS1A)
Memory address,0x226f4aca490
Stoichiometry,"S_CO2_p[C_p]_Day + S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Day + S_H2O_p[C_p]_Day --> 2.0 S_3_45_Phospho_45_D_45_glycerate_p[C_p]_Day CO2_p Day + D-Ribulose 1,5-bisphosphate Day + H2O_p Day --> 2.0 3-Phospho-D-glycerate_p Day"
GPR,ARTHCP030 or AT1G67090 or AT5G38410 or AT5G38420 or AT5G38430
Lower bound,0.0
Upper bound,1000.0


In [15]:
modelo.reactions.get_by_id('R00024_p_Night')

Reaction identifier,R00024_p_Night
Name,ribulose bisphosphate carboxylase small chain 1A / RuBisCO small subunit 1A (RBCS-1A) (ATS1A)
Memory address,0x226f55dc880
Stoichiometry,"S_CO2_p[C_p]_Night + S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Night + S_H2O_p[C_p]_Night --> 2.0 S_3_45_Phospho_45_D_45_glycerate_p[C_p]_Night CO2_p Night + D-Ribulose 1,5-bisphosphate Night + H2O_p Night --> 2.0 3-Phospho-D-glycerate_p Night"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [16]:
solution.fluxes['R00024_p_Night']

0.3374561054911554

In [17]:
solution.fluxes['R00024_p_Day']

106.77663763966915

- Rubisco + Oxigénio

In [18]:
modelo.reactions.get_by_id('R03140_p_Day')

Reaction identifier,R03140_p_Day
Name,ribulose bisphosphate carboxylase small chain 1A / RuBisCO small subunit 1A (RBCS-1A) (ATS1A)
Memory address,0x226f4e885b0
Stoichiometry,"S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Day + S_Oxygen_p[C_p]_Day --> S_2_45_Phosphoglycolate_p[C_p]_Day + S_3_45_Phospho_45_D_45_glycerate_p[C_p]_Day D-Ribulose 1,5-bisphosphate Day + Oxygen_p Day --> 2-Phosphoglycolate Day + 3-Phospho-D-glycerate_p Day"
GPR,ARTHCP030 or AT1G67090 or AT5G38410 or AT5G38420 or AT5G38430
Lower bound,0.0
Upper bound,1000.0


In [19]:
modelo.reactions.get_by_id('R03140_p_Night')

Reaction identifier,R03140_p_Night
Name,ribulose bisphosphate carboxylase small chain 1A / RuBisCO small subunit 1A (RBCS-1A) (ATS1A)
Memory address,0x226f57ead90
Stoichiometry,"S_D_45_Ribulose_32_1_44_5_45_bisphosphate_p[C_p]_Night + S_Oxygen_p[C_p]_Night --> S_2_45_Phosphoglycolate_p[C_p]_Night + S_3_45_Phospho_45_D_45_glycerate_p[C_p]_Night D-Ribulose 1,5-bisphosphate Night + Oxygen_p Night --> 2-Phosphoglycolate Night + 3-Phospho-D-glycerate_p Night"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
solution.fluxes['R03140_p_Day']

0.0

In [21]:
solution.fluxes['R03140_p_Night']

0.0

- PGK

In [24]:
modelo.reactions.get_by_id("R01512_c_Day")

Reaction identifier,R01512_c_Day
Name,"phosphoglycerate kinase, putative"
Memory address,0x226f4ccabb0
Stoichiometry,S_3_45_Phospho_45_D_45_glycerate_c[C_c]_Day + S_ATP_c[C_c]_Day <=> S_3_45_Phospho_45_D_45_glyceroyl_32_phosphate_c[C_c]_Day + S_ADP_c[C_c]_Day 3-Phospho-D-glycerate_c Day + ATP_c Day <=> 3-Phospho-D-glyceroyl phosphate Day + ADP_c Day
GPR,AT1G56190 or AT1G79550 or AT3G12780
Lower bound,-1000.0
Upper bound,1000.0


In [25]:
modelo.reactions.get_by_id("R01512_c_Night")

Reaction identifier,R01512_c_Night
Name,"phosphoglycerate kinase, putative"
Memory address,0x226f56fbdf0
Stoichiometry,S_3_45_Phospho_45_D_45_glycerate_c[C_c]_Night + S_ATP_c[C_c]_Night <=> S_3_45_Phospho_45_D_45_glyceroyl_32_phosphate_c[C_c]_Night + S_ADP_c[C_c]_Night 3-Phospho-D-glycerate_c Night + ATP_c Night <=> 3-Phospho-D-glyceroyl phosphate Night + ADP_c Night
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [28]:
solution.fluxes['R01512_c_Day']

-30.381016165838282

In [27]:
solution.fluxes['R01512_c_Night']

0.35186115965192183

- GAPDH

In [33]:
modelo.reactions.get_by_id("R01063_p_Night")

Reaction identifier,R01063_p_Night
Name,"glyceraldehyde 3-phosphate dehydrogenase, chloroplast, putative / NADP-dependent..."
Memory address,0x226f56b5700
Stoichiometry,S_NADP_43__p[C_p]_Night + S_Orthophosphate_p[C_p]_Night + S__40_2R_41__45_2_45_Hydroxy_45_3_45__40_phosphonooxy_41__45_propanal_p[C_p]_Night <=>... NADP+ Night + Orthophosphate_p Night + (2R)-2-Hydroxy-3-(phosphonooxy)-propanal_p Night <=> 3-Phospho-D-glyceroyl phosphate Night + H+_p Night + NADPH Night
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [34]:
solution.fluxes['R01063_p_Night']

0.06139859161561175

In [41]:
modelo.metabolites.get_by_id("S_D_45_Fructose_32_1_44_6_45_bisphosphate_p[C_p]_Night").reactions

frozenset({<Reaction R00762_p_Night at 0x226f5671bb0>,
           <Reaction R01068N_p_Night at 0x226f4b5b8b0>})

In [42]:
modelo.reactions.get_by_id("R00762_p_Night")

Reaction identifier,R00762_p_Night
Name,"fructose-1,6-bisphosphatase, putative / D-fructose-1,6-bisphosphate 1-phosphohydrolase, putative..."
Memory address,0x226f5671bb0
Stoichiometry,"S_D_45_Fructose_32_1_44_6_45_bisphosphate_p[C_p]_Night + S_H2O_p[C_p]_Night --> S_D_45_Fructose_32_6_45_phosphate_p[C_p]_Night + S_Orthophosphate_p[C_p]_Night D-Fructose 1,6-bisphosphate Night + H2O_p Night --> D-Fructose 6-phosphate Night + Orthophosphate_p Night"
GPR,
Lower bound,0.0
Upper bound,1000.0
